In [4]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.types as T
import pyspark
spark = SparkSession.builder \
    .master("local") \
    .appName("Time Series") \
    .getOrCreate()

23/03/11 21:24:28 WARN Utils: Your hostname, melanie-ThinkPad-X1-Carbon-Gen-9 resolves to a loopback address: 127.0.1.1; using 172.16.5.171 instead (on interface wlp0s20f3)
23/03/11 21:24:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/11 21:24:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
crime_data = spark.read.csv("/home/melanie/Desktop/GitHub/ChicagoCrime/data/Crimes_-_2001_to_Present.csv", header=True)

In [6]:
crime_data.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE|

In [7]:
crime_data = crime_data.withColumn("Count", F.lit(1))

In [8]:
crime_data.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Count|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|   

In [9]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
crime_data = crime_data.withColumn("Date/Time", F.to_timestamp(F.col("Date"), "M/d/yyyy H:mm:ss"))
crime_data.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+-------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Count|          Date/Time|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+-------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043X

In [10]:
adjustedCrimeData = crime_data.withColumn('year', F.year("Date/Time")).withColumn('month', F.month("Date/Time"))
adjustedCrimeData.show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+-------------------+-----+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|year|          Updated On|    Latitude|    Longitude|            Location|Count|          Date/Time|month|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----+-------------------+-----+
|10224738|   HY411648|09/05/2015 0

In [11]:
arrests = adjustedCrimeData.groupBy("Arrest", "year", "month").sum("Count")
arrests.show()

+------+----+-----+----------+
|Arrest|year|month|sum(Count)|
+------+----+-----+----------+
| false|2012|    1|     18749|
|  true|2019|    3|      4761|
| false|2005|    2|     20774|
|  true|2014|    9|      6757|
|  true|2015|    5|      6313|
| false|2009|    9|     24304|
|  true|2012|    6|      8047|
| false|2017|    1|     17874|
| false|2016|    7|     20241|
| false|2021|    2|     11024|
|  true|2021|    4|      2215|
| false|2017|   11|     17485|
|  true|2014|   10|      6861|
| false|2009|    8|     25858|
|  true|2022|    9|      2309|
|  true|2012|    5|      7901|
| false|2006|   12|     23954|
|  true|2014|    1|      5751|
| false|2011|    4|     21203|
|  true|2010|    4|      8705|
+------+----+-----+----------+
only showing top 20 rows



In [16]:
arrests_df = arrests.toPandas()
arrests_df.to_csv("/home/melanie/Desktop/GitHub/ChicagoCrime/data/arrests_by_month.csv")

In [12]:
adjustedCrimeData = adjustedCrimeData.filter(adjustedCrimeData["Primary Type"] == "THEFT").groupBy("year", "month").sum("Count")
adjustedCrimeData.show()

+----+-----+----------+
|year|month|sum(Count)|
+----+-----+----------+
|2022|   10|      5224|
|2015|    2|      3228|
|2019|   10|      5390|
|2001|    1|      7867|
|2017|    3|      4493|
|2008|    8|      8501|
|2017|    8|      6463|
|2014|    4|      4836|
|2005|   10|      7746|
|2020|    6|      3085|
|2019|    5|      5398|
|2017|   10|      5735|
|2018|   10|      5670|
|2021|    8|      3886|
|2016|    7|      5821|
|2015|   12|      5027|
|2016|   11|      5257|
|2019|    3|      4468|
|2012|   12|      5866|
|2012|    4|      6192|
+----+-----+----------+
only showing top 20 rows



In [13]:
import pandas as pd
adjustedCrimeDataPandas = adjustedCrimeData.toPandas()
adjustedCrimeDataPandas.head()

,year,month,sum(Count)
0,2022.0,10.0,5224
1,2015.0,2.0,3228
2,2019.0,10.0,5390
3,2001.0,1.0,7867
4,2017.0,3.0,4493


In [14]:
adjustedCrimeDataPandas.to_csv("/home/melanie/Desktop/GitHub/ChicagoCrime/data/thefts_by_month.csv")